In [5]:
pip install PyMuPDF langchain sentence-transformers pinecone-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [6]:
import requests

url = "https://www.nature.com/articles/s41597-022-01899-x.pdf?proof=t"  # PDF link from Nature page
r = requests.get(url, stream=True)
if r.status_code == 200:
    with open("mimic_iv_descriptor.pdf", "wb") as f:
        for chunk in r.iter_content(1024):
            f.write(chunk)
    print("✅ PDF downloaded as mimic_iv_descriptor.pdf")
else:
    print("❌ Failed to download PDF:", r.status_code)


✅ PDF downloaded as mimic_iv_descriptor.pdf


In [7]:
pip install pymupdf


In [8]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    all_text = ""
    for page in doc:
        all_text += page.get_text()
    return all_text

# Extract and save
pdf_path = "mimic_iv_descriptor.pdf"
raw_text = extract_text_from_pdf(pdf_path)

# Save to text file (optional)
with open("mimic_iv_descriptor.txt", "w", encoding="utf-8") as f:
    f.write(raw_text)

print("✅ Text extracted from PDF. First 500 characters:\n")
print(raw_text[:500])


✅ Text extracted from PDF. First 500 characters:

1
Scientific Data | (2023) 10:1 | https://doi.org/10.1038/s41597-022-01899-x
www.nature.com/scientificdata
MIMIC-IV, a freely accessible 
electronic health record dataset
Alistair E. W. Johnson1,2 ✉, Lucas Bulgarelli   1, Lu Shen3, Alvin Gayles3, Ayad Shammout3, 
Steven Horng   3, Tom J. Pollard   1, Sicheng Hao1, Benjamin Moody1, Brian Gow   1,  
Li-wei H. Lehman1, Leo A. Celi   1,3 & Roger G. Mark   1
Digital data collection during routine clinical practice is now ubiquitous within hospitals. 


In [9]:
from typing import List

def chunk_text(text: str, chunk_size: int = 500, overlap: int = 100) -> List[str]:
    chunks = []
    start = 0
    text_length = len(text)
    while start < text_length:
        end = min(start + chunk_size, text_length)
        chunks.append(text[start:end])
        start += chunk_size - overlap  # move forward with overlap
    return chunks

# Create chunks
chunks = chunk_text(raw_text)
print(f"✅ Created {len(chunks)} text chunks.")
print("\n🔹 First chunk preview:\n")
print(chunks[0])


✅ Created 114 text chunks.

🔹 First chunk preview:

1
Scientific Data | (2023) 10:1 | https://doi.org/10.1038/s41597-022-01899-x
www.nature.com/scientificdata
MIMIC-IV, a freely accessible 
electronic health record dataset
Alistair E. W. Johnson1,2 ✉, Lucas Bulgarelli   1, Lu Shen3, Alvin Gayles3, Ayad Shammout3, 
Steven Horng   3, Tom J. Pollard   1, Sicheng Hao1, Benjamin Moody1, Brian Gow   1,  
Li-wei H. Lehman1, Leo A. Celi   1,3 & Roger G. Mark   1
Digital data collection during routine clinical practice is now ubiquitous within hospitals. 


In [10]:
from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')  # output dimension = 384

# Generate embeddings
embeddings = model.encode(chunks, show_progress_bar=True, normalize_embeddings=True)

print(f"✅ Generated {len(embeddings)} embeddings of dimension {len(embeddings[0])}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Generated 114 embeddings of dimension 384


In [12]:
!pip install faiss-cpu



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 51.1 MB/s eta 0:00:00


In [13]:
import faiss
import numpy as np


In [14]:
# Convert embeddings to numpy array
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatIP(embedding_dim)  # Using cosine similarity (with normalized vectors)

# Add vectors
index.add(np.array(embeddings))

print(f"✅ FAISS index built with {index.ntotal} vectors.")


✅ FAISS index built with 114 vectors.


In [15]:
faiss.write_index(index, "medical_faiss.index")


In [16]:
def semantic_search(query, model, index, chunks, top_k=3):
    # Embed the query
    query_embedding = model.encode([query], normalize_embeddings=True)

    # Search the index
    D, I = index.search(np.array(query_embedding), top_k)

    print(f"\n🔍 Top {top_k} results for: '{query}'\n")
    for idx in I[0]:
        print(f"🧾 {chunks[idx]}")
        print("-" * 80)


In [17]:
query = "What are the symptoms of high blood pressure?"
semantic_search(query, model, index, chunks)



🔍 Top 3 results for: 'What are the symptoms of high blood pressure?'

🧾  of 
MIMIC-IV v2.2, the OMR table contains data for five measurements: blood pressure, height, weight, body mass 
index, and the Estimated Glomerular Filtration Rate (eGFR). These values are available from both inpatient and 
outpatient visits, and in many cases a “baseline” value from before a patient’s hospitalization is available.
Medication.  There are four tables in the hosp module which track medication prescription and administration: 
prescriptions, pharmacy, emar, and emar_detail. The p
--------------------------------------------------------------------------------
🧾 ts reported to the care providers. This information is not captured in this table, which only provides the final 
interpretation of a microbiology culture as it was documented at storetime.
Finally, the omr table provides information from the Online Medical Record (OMR) for the patient. OMR 
is a general system used for documenting patient i

In [18]:
semantic_search("what is blood pressure data used for", model, index, chunks)



🔍 Top 3 results for: 'what is blood pressure data used for'

🧾 h as blood. Multiple measurements are often taken for a single specimen, delineated by 
the specimen_id column in the labevents table and the micro_specimen_id column in microbiology-
events. For example, blood gas measurements made on the same sample will share the same specimen_id 
with one concept specifying the type of specimen (arterial, venous, etc).
Microbiology measurements are stored in a single table with columns dedicated to domain specific con-
cepts. Measurements follow a directed h
--------------------------------------------------------------------------------
🧾 1
Scientific Data | (2023) 10:1 | https://doi.org/10.1038/s41597-022-01899-x
www.nature.com/scientificdata
MIMIC-IV, a freely accessible 
electronic health record dataset
Alistair E. W. Johnson1,2 ✉, Lucas Bulgarelli   1, Lu Shen3, Alvin Gayles3, Ayad Shammout3, 
Steven Horng   3, Tom J. Pollard   1, Sicheng Hao1, Benjamin Moody1, Brian Gow   1,  
Li

In [19]:
semantic_search("how to track patient medication", model, index, chunks)




🔍 Top 3 results for: 'how to track patient medication'

🧾 conducted code reviews, and tracked issues using 
a ticket system. Code was managed using version control software. Data transformation was executed with a 
single, reproducible build script. Validation of the build process assessed data integrity, data consistency, and 
deidentification. Unit tests were created to apply these checks during development.
Fig. 4  Visualization of medication information documented within MIMIC-IV for a single patient’s 
hospitalization: hadm_id 28503629. The annota
--------------------------------------------------------------------------------
🧾 ion related tables, emar and emar_detail, are sourced from the electronic Medicine 
Administration Record (eMAR). The eMAR system requires barcode scanning of a patient wristband and 
the medication at the time of administration and was deployed throughout the BIDMC between 2014–2016. 
By 2016, all units of the hospital had the eMAR system deployed, and t

In [20]:
semantic_search("what is omr table in mimic", model, index, chunks)


🔍 Top 3 results for: 'what is omr table in mimic'

🧾 ts reported to the care providers. This information is not captured in this table, which only provides the final 
interpretation of a microbiology culture as it was documented at storetime.
Finally, the omr table provides information from the Online Medical Record (OMR) for the patient. OMR 
is a general system used for documenting patient information from visits at BIDMC affiliated institutes. As of 
MIMIC-IV v2.2, the OMR table contains data for five measurements: blood pressure, height, weigh
--------------------------------------------------------------------------------
🧾  of 
MIMIC-IV v2.2, the OMR table contains data for five measurements: blood pressure, height, weight, body mass 
index, and the Estimated Glomerular Filtration Rate (eGFR). These values are available from both inpatient and 
outpatient visits, and in many cases a “baseline” value from before a patient’s hospitalization is available.
Medication.  There are fou